##### This code produces Prophet forecast for the test period.

In [1]:
# import libararies
import pandas as pd
import numpy as np
import os
from fbprophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [2]:
# read data sets 
data_test = pd.read_csv('Estimar2.txt', sep='|', low_memory=False)
data_train = pd.read_csv('Modelar_UH2021.txt', sep='|', low_memory=False)

# select columns needed for further time series modelling
data = data_train.loc[:,['fecha','id','visitas','dia_atipico','campaña','unidades_vendidas']]
data_t = data_test.loc[:,['fecha','id','visitas','dia_atipico','campaña']]

# eliminate duplicates
data = data.drop_duplicates()

# change dates data type to 'timestamps' 
data['fecha']=pd.to_datetime(data['fecha'], format="%d/%m/%Y %H:%M:%S")

# unique stock codes in the test dataset
codes_test = data_test['id'].unique()
len(codes_test)

2736

In [3]:
# create and save separate files for each stock code to be predicted
# creamos y guardamos los archivos separados para los articulos que debemos pronosticar
try:  
    os.mkdir('./files')  
except OSError as error:  
    print('Directory already exists') 

for each in codes_test:
    df = data[data['id']==each].copy()
    path = './files/' +str(each)+'.csv'
    df.to_csv(path, index=False)

Directory already exists


In [4]:
# function for data preprocessing
def prep_data (stock_code):
    
    # read previously saved labeled data for an individual stock code
    path = './files/' +str(stock_code)+'.csv'   
    df = pd.read_csv(path)
    
    # read unlabeled data for the stock code in the test set
    df_t = data_t[data_t['id']==stock_code].copy()
    
    
    # mark duplicates to eliminate in train data sets
    df.loc[:,'delete']=0
    
    fechas_dup = pd.Series(df['fecha'])[pd.Series(df['fecha']).duplicated()].values
    for fecha in fechas_dup:
        index = df[(df['fecha'] ==fecha ) & (df['campaña']==0)].index
        df.loc[index, 'delete']=1

    
    # eliminate duplicates
    index_names = df[df['delete'] == 1 ].index 
    df.drop(index_names, inplace = True)
    

    # change index to timestamp
    df.index=df.loc[:,'fecha']
    df.index = pd.to_datetime(df.index)

    df_t.index=df_t.loc[:,'fecha']
    df_t.index = pd.to_datetime(df_t.index)
    

    # for each day of promotion ('campaña'=1), change'dia_atipico' to zero to have these codes independent 
    for i in range(len(df)):
        if (df.loc[df.index[i],'campaña']==1):
            df.loc[df.index[i],['dia_atipico']]=0   

    for i in range(len(df_t)):
        if (df_t.loc[df_t.index[i],'campaña']==1):
            df_t.loc[df_t.index[i],['dia_atipico']]=0              
            
            
    # eliminate the columns which are not required
    df = pd.DataFrame(df[['unidades_vendidas','visitas','campaña','dia_atipico']])
    df_t = pd.DataFrame(df_t[['visitas','campaña','dia_atipico']])

   
    # create ts and add columns required by Prophet
    ts = df.copy()   
    ts['ds']=ts.index
    ts['y']=ts.loc[:,'unidades_vendidas']
    
    return (ts, df_t)


In [8]:
def forecast_prophet(stock_code,ts, df_t, start_date='2016-10-01', periods=92, scale=0.05):

    # create a time series for regressors
    columns = ['dia_atipico','campaña', 'visitas']
    indx = pd.date_range(start_date, periods=periods, freq='D')
    df = pd.DataFrame(index=indx)
    df_ = pd.merge(left=df, right=df_t, how='left', left_on=df.index, right_on=df_t.index)
    df_.fillna(value=0, inplace=True)
    
    ts_t = pd.concat([ts[columns], df_[columns]])

    # make and fit a model with three additional regressors
    m = Prophet(changepoint_prior_scale=scale)
    m.add_regressor('campaña')
    m.add_regressor('dia_atipico')
    m.add_regressor('visitas')
    m.fit(ts)

    # make predictions
    future = m.make_future_dataframe(periods=periods, freq='D')
    future.loc[:,'campaña']=ts_t.loc[:,'campaña'].to_list()
    future.loc[:,'dia_atipico']=ts_t.loc[:,'dia_atipico'].to_list()
    future.loc[:,'visitas']=ts_t.loc[:,'visitas'].to_list()
    forecast = m.predict(future)
       
    # extract predictions for test period
    predictions = forecast.iloc[-periods:][['ds','yhat']]
    predictions['yhat']=predictions['yhat'].astype(int)
    
    
    # change negative values to zeros
    for i in range(len(predictions)):
        if predictions.iloc[i,1] < 0:
            predictions.iloc[i,1] = 0
    
    # create dataframe for predictions
    pred_lines = pd.DataFrame(columns=['fecha','id','unidades_vendidas'])
    pred_lines['unidades_vendidas']=predictions['yhat']
    pred_lines['fecha']=predictions['ds']
    pred_lines['id']=stock_code          
    
    return (pred_lines)

In [12]:
#### FORECASTING WITH PROPHET FOR TEST PERIOD

# Set dictionary for keeping forecasts
pred_prophet = pd.DataFrame()

for each in codes_test:

    ts, df_t = prep_data(each)

    pred_lines = forecast_prophet(each, ts, df_t, start_date='2016-10-01', periods=92, scale=0.05)

    # add predictions to the dataframe
    pred_prophet= pd.concat([pred_prophet, pred_lines])

    # print progress line
    print('Item number: '+str(np.where(codes_test==each)[0][0]+1) + ', Item code: '+ str(each))


# save forecast to file
pred_prophet.to_csv('prophet_pred_updated.csv', index=False)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
C:\Users\Tanya\anaconda3\lib\site-packages\pystan\misc.py:399: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(np.asarray(v).dtype, float):
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Item number: 1, Item code: 21972
Item number: 2, Item code: 23910


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Item number: 3, Item code: 27144


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

Item number: 2733, Item code: 457416


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Item number: 2734, Item code: 457422


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Item number: 2735, Item code: 458650
Item number: 2736, Item code: 458660


In [20]:
# this code adjust forecast to the structure of the file 'Estimar_UH2021.txt': duplicates and eliminates some prediction lines
# and saves in the format required for the competition

df = data_t[['fecha', 'id']].copy()
df['fecha']=pd.to_datetime(df['fecha'], format='%Y-%m-%d')
df.index=df[['fecha','id']]


prophet = pd.read_csv('prophet_pred_updated.csv', parse_dates=True)
prophet['fecha']=pd.to_datetime(prophet['fecha'], format='%Y-%m-%d')
prophet.index = prophet[['fecha', 'id']]
prophet.drop(['fecha', 'id'], axis=1, inplace=True)

merged = pd.merge(left=df, right=prophet, how='left', left_on=df.index, right_on=prophet.index)
merged.drop('key_0', axis=1, inplace=True)

prophet_pred_atmira=merged.copy()
prophet_pred_atmira.rename(columns={'fecha': 'FECHA','id':'ID','unidades_vendidas': 'UNIDADES'}, inplace=True)
prophet_pred_atmira['FECHA'] = prophet_pred_atmira['FECHA'].dt.strftime('%d/%m/%Y')

prophet_pred_atmira.to_csv('Atmira_Universidad Internacional de Valencia_Heuralia_updated.txt', sep='|', index=False)

In [13]:
pred_prophet

,fecha,id,unidades_vendidas
487,2016-10-01,21972,2
488,2016-10-02,21972,3
489,2016-10-03,21972,1
490,2016-10-04,21972,0
491,2016-10-05,21972,0
...,...,...,...
574,2016-12-27,458660,8
575,2016-12-28,458660,9
576,2016-12-29,458660,6
577,2016-12-30,458660,10


In [17]:
atmira = pd.read_csv('Atmira_Universidad Internacional de Valencia_Heuralia_updated.txt', sep='|')

In [19]:
atmira.describe()

,ID,UNIDADES
count,212841.000000,212841.000000
mean,254764.373124,7.296113
std,97368.478179,51.367326
min,21972.000000,0.000000
25%,177232.000000,0.000000
50%,264804.000000,1.000000
75%,335178.000000,5.000000
max,458660.000000,12420.000000
